In [2]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip


--2025-09-20 20:28:39--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 54.231.166.177, 52.216.92.219, 52.216.44.25, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|54.231.166.177|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  18.7MB/s    in 14s     

2025-09-20 20:28:53 (17.2 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]

Archive:  annotations_trainval2017.zip
replace annotations/instances_train2017.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: annotations/instances_train2017.json  
y

replace annotations/instances_val2017.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: error:  invalid response [{ENTER}]
replace annotations/instances_val2017.json? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating:

In [3]:
train_captions = "/content/annotations/captions_train2017.json"


In [5]:
import json

In [8]:
with open(train_captions, "r") as f:
    data = json.load(f)

print(data.keys())
print(len(data["annotations"]))
print(data["annotations"][0].values())

dict_keys(['info', 'licenses', 'images', 'annotations'])
591753
dict_values([203564, 37, 'A bicycle replica with a clock as the front wheel.'])


In [21]:
captions = []

for ann in data["annotations"][:]:
    caption = ann["caption"].lower()
    captions.append(caption)



In [22]:
print(len(captions))
print(captions[:5])

591753
['a bicycle replica with a clock as the front wheel.', 'a room with blue walls and a white sink and door.', 'a car that seems to be parked illegally behind a legally parked car', 'a large passenger airplane flying through the air.', 'there is a gol plane taking off in a partly cloudy sky.']


In [23]:
import spacy

In [24]:
nlp = spacy.load("en_core_web_sm")

In [29]:
tokenized_captions = []

for caption in captions:
    doc = nlp(caption)
    tokens = [token.text for token in doc]
    tokenized_captions.append(tokens)


In [32]:
captions[1]

'a room with blue walls and a white sink and door.'

In [31]:
tokenized_captions[1]

['a',
 'room',
 'with',
 'blue',
 'walls',
 'and',
 'a',
 'white',
 'sink',
 'and',
 'door',
 '.']

In [33]:
with open("tokenized_captions.json", "w") as f:
    json.dump(tokenized_captions, f)

In [34]:
import pickle

# Save
with open("tokenized_captions.pkl", "wb") as f:
    pickle.dump(tokenized_captions, f)

# Load
with open("tokenized_captions.pkl", "rb") as f:
    tokenized_captions = pickle.load(f)
